# Vorbereitung, Importe

In [1]:
import fitz  # PyMuPDF
import os
import pandas as pd
import matplotlib.pyplot as plt #nur für Testzwecke
import re #Regular Expressions importieren, um nach Wortstämmen zu suchen

In [2]:
import json

# PDF einlesen, leicht vorsäubern und in Datenbank packen

In [3]:
list_of_fulltext_ubs = []
years = [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

#Doppelter for-Loop, um jede PDF-Seite in jedem Bericht auszulesen

for year in years:
    file_path = f"{year}_ubs_Review_D.pdf"
    
    if os.path.exists(file_path):
        ubs_file = fitz.open(file_path)
        complete_text_ubs = ""

        for page_num in range(ubs_file.page_count):
            page = ubs_file.load_page(page_num)
            text = page.get_text()
            #Mit Regular Expressions und Replace-Methode mehrfache Leerzeichen, Zeilenumbrüche und Satzzeichen wegbringen
            text_without_numbers = re.sub(r'\d+', '', text)
            text_without_linebreaks = text_without_numbers.replace("\n", " ")
            text_without_linebreaks = text_without_linebreaks.replace(",", " ")
            text_without_linebreaks = text_without_linebreaks.replace(".", " ")
            text_without_linebreaks = text_without_linebreaks.replace("()", " ")
            text_without_linebreaks = text_without_linebreaks.replace("«", "")
            text_without_linebreaks = text_without_linebreaks.replace("»", "")
            text_without_linebreaks = text_without_linebreaks.replace(":", "")
            text_without_linebreaks = re.sub(r"\s+", " ", text_without_linebreaks).strip()
            text_without_linebreaks.split(" ")
            #Über das Zeilenende hinweg getrennte Wörter wieder zusammenfügen
            text_without_linebreaks = text_without_linebreaks.replace("- ", "", )
            complete_text_ubs += text_without_linebreaks

        list_of_fulltext_ubs.append([year, complete_text_ubs])

        ubs_file.close()
    else:
        print(f"File not found for year {year}")


In [4]:
# Pandas-Dataframe erstellen für die Weiterbearbeitung
df = pd.DataFrame(list_of_fulltext_ubs, columns=['Year', 'Text'])
#df

# Wortliste erstellen

In [5]:
def create_word_list(text):
    return text.split()

In [6]:
df['Wortliste'] = df['Text'].apply(create_word_list)

In [7]:
# Zwischenspeicherung als CSV - war nötig, um Fehler beim weiteren Filtern erkennen zu können
df.to_csv('ubs_Review_volltext.csv', index=False)

print("DataFrame saved to ubs_Review_volltext.csv")

DataFrame saved to ubs_Review_volltext.csv


# Nutzlose Wörter rausfiltern

In [8]:
def remove_stopwords(word_list, stopwords):
        return [word for word in word_list if word.lower() not in stopwords]

In [9]:
# Stopwords reinladen, Liste dieser Wörter erstellen. Achtung: Immer neuestes Textfile verwenden!
with open('stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

In [10]:
df['Wortliste_2'] = df['Wortliste'].apply(lambda x: remove_stopwords(x, stopwords))

# Zahlen rausfiltern

In [11]:
def zahlenfiltern(word_list):
        return [word for word in word_list if not word.isdigit()]

In [12]:
df['Wortliste_2'] = df['Wortliste_2'].apply(lambda x: zahlenfiltern(x))

# Kommas und Punkte am Wortende rauswerfen

In [13]:
df['Wortliste_3'] = df['Wortliste_2'].apply(lambda word_list: [word.rstrip(',.:').rstrip('-') for word in word_list])
#Hier hat es eine gewisse Doppelung zum Filterschritt im For-Loop drin. Ich behielt den Schritt sicherheitshalber drin.

# Länge der jeweiligen Jahresberichte ergänzen

In [14]:
# Zusätzliche Spalte im Dataframe mit der Gesamtlänge des jeweiligen Berichts einfügen, um nachher Häufigkeit zu berechnen
df['Total_Length'] = df['Wortliste_3'].apply(len)

In [2]:
#df.head(20)

# Zählen, wie oft meine Unterbegriffe vorkommen

In [15]:
# Pfad der Suchbegriffe-Datei festlegen
suchbegriffe_file_path = 'suchbegriffe.txt'

# Suchbegriffe-Dictionary aus der Datei auslesen
with open(suchbegriffe_file_path, 'r', encoding='utf-8') as file:
    suchbegriffe_dict = json.load(file)

In [16]:
# Erstelle ein neues Dataframe für die Auswertung
result_df = pd.DataFrame(columns=['Year'])

# Iteriere durch jedes Jahr im Original-Dataframe
for year in df['Year'].unique():
    # Extrahiere die relevanten Zeilen für das aktuelle Jahr
    year_rows = df[df['Year'] == year]

    # Leeres Dictionary für die Häufigkeiten des aktuellen Jahres
    year_freqs = {'Year': year}

    # Iteriere durch die Suchbegriffe und zähle deren Häufigkeit im aktuellen Jahr
    for key, value in suchbegriffe_dict.items():
        # Initialisiere ein leeres Dictionary für die Unterbegriffe
        subfreqs = {}
        
        # Iteriere durch die Unterbegriffe und zähle deren Häufigkeit im aktuellen Jahr
        for subterm in value:
            sub_count = sum(year_rows['Wortliste_3'].apply(lambda words: sum(word.lower() == subterm for word in words)))
            subfreqs[subterm] = sub_count
        
        # Füge die Häufigkeiten der Unterbegriffe zum Ergebnis-Dataframe hinzu
        year_freqs.update(subfreqs)
    
    # Füge die Häufigkeiten des aktuellen Jahres zum Ergebnis-Dataframe hinzu
    result_df = pd.concat([result_df, pd.DataFrame([year_freqs])])

# Index des Ergebnis-Dataframes resetten
result_df = result_df.reset_index(drop=True)

# Wordcount der Unterbegriffe aggregieren

In [17]:
# Erstelle ein leeres Dataframe für die Oberbegriffe
oberbegriffe_df = pd.DataFrame(columns=list(suchbegriffe_dict.keys()), dtype=int)

# Iteriere durch jedes Jahr im Original-Dataframe
for year in df['Year'].unique():
    # Extrahiere die relevanten Zeilen für das aktuelle Jahr
    year_rows = df[df['Year'] == year]

    # Initialisiere ein leeres Dictionary für die Oberbegriffe
    oberbegriffe_freqs = {key: 0 for key in suchbegriffe_dict.keys()}

    # Iteriere durch die Spalten von result_df
    for column in result_df.columns:
        # Iteriere durch die Unterbegriffe des aktuellen Oberbegriffs
        for key, value in suchbegriffe_dict.items():
            if column in value:
                # Füge den Wordcount dem entsprechenden Oberbegriff hinzu
                oberbegriffe_freqs[key] += result_df.loc[result_df['Year'] == year, column].values[0]

    # Füge die Häufigkeiten des aktuellen Jahres zu oberbegriffe_df hinzu
    oberbegriffe_df = pd.concat([oberbegriffe_df, pd.DataFrame([oberbegriffe_freqs])], ignore_index=True)

# Füge die Jahre als eigene Spalte zu oberbegriffe_df hinzu
oberbegriffe_df['Year'] = df['Year'].unique()

# Ordne die Spalten in oberbegriffe_df neu an
oberbegriffe_df = oberbegriffe_df[['Year'] + list(suchbegriffe_dict.keys())]

# Ergebnis anzeigen
#print(oberbegriffe_df)


In [21]:
#Füge die Gesamtlänge der jeweiligen Jahresberichte auch im Oberbegriffe-Dataframe hinzu
oberbegriffe_df["Total_Length"]=df["Total_Length"]

# Häufigkeiten berechnen

In [22]:
# Erstelle ein leeres Dataframe für die relative Häufigkeit der Oberbegriffe
oberbegriffe_häufigkeit_df = oberbegriffe_df.copy()

# Iteriere durch jedes Jahr im Dataframe
for year in oberbegriffe_df['Year']:
    # Extrahiere die Total Length des aktuellen Jahres
    total_length = oberbegriffe_df.loc[oberbegriffe_df['Year'] == year, 'Total_Length'].values[0]
    
    # Iteriere durch die Oberbegriffe
    for key in suchbegriffe_dict.keys():
        # Berechne die relative Häufigkeit und setze den Wert in das neue Dataframe
        oberbegriffe_häufigkeit_df.loc[oberbegriffe_häufigkeit_df['Year'] == year, key] /= total_length

# Ergebnis anzeigen
print(oberbegriffe_häufigkeit_df)


    Year   Schweiz    Global    Risiko  Nachhaltigkeit     Klima  Diversity  \
0   1999  0.007607  0.008192  0.002487        0.000585  0.000000   0.000439   
1   2000  0.009271  0.005343  0.002671        0.000471  0.000000   0.001257   
2   2001  0.006513  0.003704  0.002937        0.000383  0.000000   0.001533   
3   2002  0.005114  0.006191  0.003634        0.000808  0.000000   0.001077   
4   2003  0.005807  0.004051  0.004321        0.000270  0.000000   0.001215   
5   2004  0.004755  0.002804  0.005365        0.000488  0.000000   0.000975   
6   2005  0.006771  0.003385  0.002176        0.001451  0.000000   0.000725   
7   2006  0.005259  0.006206  0.002524        0.000631  0.000000   0.001052   
8   2007  0.005457  0.008003  0.003274        0.002546  0.001091   0.000364   
9   2008  0.003336  0.005003  0.008339        0.001334  0.000000   0.000000   
10  2009  0.007251  0.003296  0.007251        0.000659  0.000000   0.000000   
11  2010  0.005053  0.005255  0.003840        0.0006

# CSV-Export für die grafische Auswertung

In [23]:
# Dataframe exportieren in eine CSV-Datei, für die weitere Analyse und grafische Auswertung
oberbegriffe_häufigkeit_df.to_csv('Häufigkeit_UBS_Begriffe.csv', index=False)

print("Updated DataFrame saved to Häufigkeit_UBS_Begriffe.csv")

Updated DataFrame saved to Häufigkeit_UBS_Begriffe.csv
